In [ ]:
# tests.ipynb
import numpy as np
import matplotlib.pyplot as plt

from qdc.diffuser.diffuser_sim import DiffuserSimulation, Field
from qdc.diffuser.utils import phase_screen_diff, ft2, ift2
from qdc.diffuser.diffuser_result import DiffuserResult

# -------------
# 1) Setup simulation
# -------------
sim = DiffuserSimulation(
    Nx=256, Ny=256,
    Lx=2e-3, Ly=2e-3,
    lam_center=808e-9,
    lam_half_range=40e-9,
    Nwl=5,
    waist=20e-6,
    focal_length=100e-3,
    diffuser_angle=0.5
)

# -------------
# 2) Initial Gaussian at detection
# -------------
field_det = sim.make_detection_gaussian(sim.lam_center)
I_det = np.abs(field_det.E)**2

fig, ax = plt.subplots()
im = ax.imshow(I_det, extent=[sim.x[0]*1e6, sim.x[-1]*1e6, sim.y[0]*1e6, sim.y[-1]*1e6],
               cmap='viridis', origin='lower')
fig.colorbar(im, ax=ax, label='Intensity')
ax.set_title("Initial Gaussian at Detection Plane (center WL)")
ax.set_xlabel("x [µm]")
ax.set_ylabel("y [µm]")
plt.show()

# -------------
# 3) Back-propagate to crystal plane
# -------------
field_crystal = sim.make_detection_gaussian(sim.lam_center)
field_crystal = sim.__class__.backprop_farfield_fft(field_crystal, sim.f)  # or use the function from utils
I_crystal = np.abs(field_crystal.E)**2

fig, ax = plt.subplots()
im = ax.imshow(I_crystal, extent=[sim.x[0]*1e6, sim.x[-1]*1e6, sim.y[0]*1e6, sim.y[-1]*1e6],
               cmap='viridis', origin='lower')
fig.colorbar(im, ax=ax, label='Intensity')
ax.set_title("Field Intensity at Crystal Plane")
ax.set_xlabel("x [µm]")
ax.set_ylabel("y [µm]")
plt.show()

# -------------
# 4) Single diffuser phase at lam_center
# -------------
phase_ref = phase_screen_diff(sim.x, sim.y, sim.lam_center, sim.diffuser_angle)

fig, ax = plt.subplots()
pcm = ax.imshow(phase_ref, extent=[sim.x[0]*1e3, sim.x[-1]*1e3, sim.y[0]*1e3, sim.y[-1]*1e3],
                cmap='twilight', origin='lower')
fig.colorbar(pcm, ax=ax, label='Phase [rad]')
ax.set_title("Single Diffuser Phase (lam_center)")
ax.set_xlabel("x [mm]")
ax.set_ylabel("y [mm]")
plt.show()

# -------------
# 5) k-space amplitude after applying diffuser (for a given WL)
#    Let's pick lam = lam_center for demonstration
# -------------
lam = sim.lam_center
sc_field = Field(sim.x, sim.y, lam, field_crystal.E.copy())

# scale the diffuser phase for this lam = lam_center => scale=1
phase_lam = (sim.lam_center / lam) * phase_ref
sc_field.E *= np.exp(1j * phase_lam)

# Now do an ft2 to see k-space amplitude
from qdc.diffuser.utils import ft2
G, f_x, f_y = ft2(sc_field.E, sc_field.x, sc_field.y)
Ik = np.abs(G)**2

fig, ax = plt.subplots()
im = ax.imshow(Ik, origin='lower', cmap='inferno')
fig.colorbar(im, ax=ax, label='|G(kx,ky)|^2')
ax.set_title("k-space amplitude after diffuser")
ax.set_xlabel("freq_x [index]")
ax.set_ylabel("freq_y [index]")
plt.show()

# -------------
# 6) Forward-propagate to detection => final speckle at that wavelength
# -------------
from qdc.diffuser.utils import prop_farfield_fft
final_field = prop_farfield_fft(sc_field, sim.f)
I_speckle = np.abs(final_field.E)**2

fig, ax = plt.subplots()
im = ax.imshow(I_speckle, cmap='plasma', origin='lower',
               extent=[final_field.x[0]*1e3, final_field.x[-1]*1e3,
                       final_field.y[0]*1e3, final_field.y[-1]*1e3])
fig.colorbar(im, ax=ax, label='Intensity')
ax.set_title(f"Speckle at Detection (WL={lam*1e9:.1f}nm)")
ax.set_xlabel("x [mm]")
ax.set_ylabel("y [mm]")
plt.show()

# -------------
# 7) Full Incoherent Sum Over All Wavelengths
# -------------
I_sum = sim.run_simulation()  # uses the single diffuser approach inside
res = DiffuserResult(intensity_map=I_sum, wavelengths=None)

fig, ax = plt.subplots()
im = ax.imshow(I_sum, origin='lower',
               extent=[sim.x[0]*1e3, sim.x[-1]*1e3, sim.y[0]*1e3, sim.y[-1]*1e3],
               cmap='magma')
fig.colorbar(im, ax=ax, label='Intensity')
ax.set_title("Incoherent Sum of Speckle (All WLs)")
ax.set_xlabel("x [mm]")
ax.set_ylabel("y [mm]")
plt.show()

print("Final result:", res)
print("Contrast (full frame):", res.compute_contrast())
